살짝 튜닝해본 모델에다가 카톡대화내역 불러오기 => 긍정부정 판다.

In [1]:
# !pip install konlpy

In [2]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
# test_file = tf.keras.utils.get_file( 'test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
train_text = open(train_file,'rb').read().decode(encoding='utf-8')
# test_text = open(test_file,'rb').read().decode(encoding='utf-8')


In [3]:
import pandas as pd

df_train = pd.DataFrame({
    'feature':[ row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0 ],
    'target': [ int(row.split('\t')[2]) for row in train_text.split('\n')[1:] if row.count('\t')>0]
})

#데이터 크기 지정
df_train = df_train[::6]

In [4]:
#문장 추출
texts= [ ]
for i in df_train['feature']:
    texts.append(i)
len(texts)

25000

In [5]:
#문자열이 아닌게 있는지 확인
for i in texts:
    if type(i)!=str:
        print(i)

In [6]:
# train 데이터 입력값(X)을 정제(Cleaning)
import re
from soynlp.normalizer import repeat_normalize

def clean_korean_text(text):
    # 특수 문자 및 숫자 제거
    text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', text)
    # 반복되는 자음, 모음 제거 (e.g., 'ㅋㅋㅋ' -> 'ㅋ')
    text = repeat_normalize(text, num_repeats=1)
    # 띄어쓰기 정규화 (연속된 공백 문자를 하나의 공백 문자로 변환)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

clean_texts=[]
for i in texts:
    clean_texts.append(clean_korean_text(i))

In [7]:
# len(clean_texts)
clean_texts[:5]

['아 더빙 진짜 짜증나네요 목소리',
 '원작의 긴장감을 제대로 살려내지못했다',
 '울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해',
 '이건 정말 깨알 캐스팅과 질퍽하지않은 산뜻한 내용구성이 잘 버무러진 깨알일드',
 '주제는 좋은데 중반부터 지루하다']

In [8]:
#tfidf하면서 토큰화는 okt.morphs로 한다.
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

vectorizer=TfidfVectorizer( tokenizer= Okt().morphs)
X = vectorizer.fit_transform(clean_texts)
y = df_train.iloc[:,1]


c:\Users\uk246\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [9]:
#불균형데이터확인
df_train[df_train.iloc[:,1]==0]  #18678 
df_train[df_train.iloc[:,1]==1]  #18822
df_train.shape

(25000, 2)

In [10]:
import numpy as np
X_train_dense = np.expand_dims( X.toarray(), axis=-1 )
# X_train_dense.shape
# X.shape[1]
# X.toarray().shape[1]

In [11]:
#DNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization , Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    # input_layer,
    Input(shape=(X_train_dense.shape[1],)),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.6),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.6),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일 및 학습
optimizer= Adam(learning_rate=0.002)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 콜백 정의
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

# model.fit(X.toarray(), y, epochs = 10, batch_size = 500, validation_split=0.2)  #0.8417
model.fit(
    X.toarray(), y, 
    epochs = 20, batch_size = 50, 
    validation_split=0.3,
    callbacks=[early_stopping]
    ) 

#Okt()토크나이저를 넣지 않았을 때
# loss: 0.1659 - accuracy: 0.9387 - val_loss: 0.5841 - val_accuracy: 0.7403
#okt() 토크나이저를 썼을 때
# loss: 0.1598 - accuracy: 0.9409 - val_loss: 0.5469 - val_accuracy: 0.8003


c:\Users\uk246\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


Epoch 1/20
350/350 [==============================] - 25s 62ms/step - loss: 0.6268 - accuracy: 0.6834 - val_loss: 0.5741 - val_accuracy: 0.7781
Epoch 2/20
350/350 [==============================] - 18s 51ms/step - loss: 0.3839 - accuracy: 0.8328 - val_loss: 0.4116 - val_accuracy: 0.8077
Epoch 3/20
350/350 [==============================] - 16s 44ms/step - loss: 0.2566 - accuracy: 0.8975 - val_loss: 0.4764 - val_accuracy: 0.7919
Epoch 4/20
350/350 [==============================] - 14s 41ms/step - loss: 0.1981 - accuracy: 0.9245 - val_loss: 0.5036 - val_accuracy: 0.7939
Epoch 5/20
350/350 [==============================] - 15s 44ms/step - loss: 0.1679 - accuracy: 0.9386 - val_loss: 0.5426 - val_accuracy: 0.7953


In [12]:
# model.save('3_31_DNN 튜닝모델.h5')

In [13]:
test_texts = ['이거 정말 잼따. ', '아니 무슨 이딴게 다있냐;;','ㅋ']

#문자열이 아닌게 있는지 확인
for i in test_texts:
    if type(i)!=str:
        print(i)
        
# train 데이터 입력값(X)을 정제(Cleaning)
import re
from soynlp.normalizer import repeat_normalize

def clean_korean_text(text):
    # 특수 문자 및 숫자 제거
    text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', text)

    # 반복되는 자음, 모음 제거 (e.g., 'ㅋㅋㅋ' -> 'ㅋ')
    text = repeat_normalize(text, num_repeats=1)

    # 띄어쓰기 정규화 (연속된 공백 문자를 하나의 공백 문자로 변환)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

clean_test_texts=[]
for i in test_texts:
    clean_test_texts.append(clean_korean_text(i))

# 학습된 TF-IDF 벡터라이저로 테스트 데이터 변환
X_test_sample = vectorizer.transform(clean_test_texts)
X_test_sample_dense = X_test_sample.toarray()

# 모델을 사용하여 예측 수행
predictions = model.predict(X_test_sample_dense)
print(predictions)

# 예측 결과 출력 (긍정: 1, 부정: 0)
print("예측 결과:", np.round( predictions ).astype(int))

1/1 [==============================] - 0s 266ms/step
[[0.6815217 ]
 [0.01179763]
 [0.6967079 ]]
예측 결과: [[1]
 [0]
 [1]]


닉네임기준으로 
카톡대화내용을 기준으로 테스트 문장을 여러개 불러와서 예측 후 그 값의 비율을 계산 출력해준다.

In [14]:
# 카톡대화 불러와서 정제,(정규화코드)하는 함수
import re

def get_from_txt(txt):
    data= open(txt,"r", encoding='utf-8').read().split('\n')
    sentences=[]
    for line in data:
        pattern = r'\[(.*?)\]\s+\[(.*?)\]\s+(.+)'
        match = re.match(pattern, line)
        if match:
            name = match.group(1)  # 첫 번째 대괄호 안의 단어 추출
            time = match.group(2)  # 두 번째 대괄호 안의 단어 추출
            content = match.group(3)  # 대괄호 뒤의 내용 추출
            # print(name, time, content)
            temp=[name,time,content]
            sentences.append(temp)    
    return sentences

#함수확인
get_from_txt('sample.txt')

[['김찬란', '오후 10:01', '사진'],
 ['김찬란', '오후 9:38', 'https://youtube.com/watch?v=o1G7DWFFHso&feature=share'],
 ['김찬란', '오후 10:42', '사진'],
 ['김찬란', '오후 10:42', '사기치고다니는 한장규'],
 ['김하영', '오후 10:45', '이야'],
 ['김하영', '오후 10:46', '게임에서도 치네'],
 ['김찬란', '오후 10:46', '진짜 니말이 맞다 진짜 친구면 친구한테 민폐되기 싫어서 돈빌려달란 소리도 안한다'],
 ['김하영', '오전 12:25', '이모티콘'],
 ['김하영', '오전 12:35', '회식끝나고 집가는중'],
 ['김찬란', '오전 12:35', '음음 초밥집 회식은 어디서 하냐'],
 ['김하영', '오전 12:35', '우리 실장형네 직이 고기집해서 글로갔다'],
 ['김하영', '오전 12:36', '회식하면 맨날 돼지고기만 먹는다'],
 ['김찬란', '오전 12:36', 'ㅋㅋㅋㅋㅋㅋ케'],
 ['김하영', '오전 12:36', '이모티콘'],
 ['김찬란', '오전 12:36', '아 내수용 회식'],
 ['김찬란', '오전 12:36', '사진'],
 ['김하영', '오전 12:36', 'ㅋㅋㅋㅋ 그런거 아니고 애들이 여기가 맛있다고 여기서먹음'],
 ['김찬란', '오전 12:37', '고기 얘시하니까'],
 ['김찬란', '오전 12:37', '나도 고기가 먹고 싶군'],
 ['김하영', '오전 12:37', 'ㅋㅋㅋㅋ '],
 ['김찬란', '오전 12:37', '낼 사먹어야겠다'],
 ['김하영', '오전 12:37', '와라 엉아가 소고기 사줄께'],
 ['김찬란', '오전 12:37', '내가 사야지 씹새야 입벌려'],
 ['김찬란', '오전 12:37', '고기 들어간'],
 ['김찬란', '오전 12:38', '너는 내가 사주고 싶으니까 '],
 ['김찬란', '오전 12:38', '취직해서

In [15]:
#닉네임 입력단 
target_name = str(input())
print(target_name)  #김하영 입력

In [16]:
# 저장된 모델 불러오기
# from tensorflow.keras.models import load_models
# model= load_models('3_31_DNN 튜닝모델.h5')


#입력된 이름의 '대화내역만' 담기
received_texts= []
for i in get_from_txt('sample.txt'): 
    if i[0] == target_name:
        received_texts.append( i[2] )
        
# 이모티콘, 사진, 샵검색 제거 
clean1_received_texts = []
for i in received_texts:
    if '샵검색:' not in i: 
        if "이모티콘" not in i:
            if '샵검색:' not in i:
                clean1_received_texts.append(str(i))

#=========== 이쯤에서 답장시간 계산기 구현   =======================
          
          
#텍스트 정제  
clean2_received_texts= [clean_korean_text(i) for i in clean1_received_texts]

# # 학습된 TF-IDF 벡터라이저로 테스트 데이터 변환
X_received_texts = vectorizer.transform(clean2_received_texts)
X_received_texts_dense = X_received_texts.toarray()


# # 모델을 사용하여 예측 수행
predictions = np.round(model.predict(X_received_texts_dense))
#예측 결과 출력 (긍정: 1, 부정: 0)
print(predictions.squeeze().tolist()) #차원을 축소하고 리스트객체로 변환

cnt0=0;cnt1=0
for i in predictions.squeeze().tolist():
    if int(i)==0:
        cnt0+=1
    else:
        cnt1+=1

#부정과 긍정문의 갯수
print('부정문과 긍정문의 갯수:',cnt0,cnt1)

#긍정과 부정의 비율( 긍정문의 수 / 부정문의 수)
print('긍정과 부정의 비율: ',cnt1/cnt0)
#숫자가 1이상이고 높을수록 긍정적이다.



ValueError: Found array with 0 sample(s) (shape=(0, 35800)) while a minimum of 1 is required by TfidfTransformer.